In [1]:
import pandas as pd
import os
import string
import contractions
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sean\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sean\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sean\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sean\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("web_scraping.csv", index_col = 0)

In [4]:
pd.set_option('max_colwidth',150)
df.head()

,date,drug_name,condition,rating,comment
0,"March 21, 2019",Abacavir,HIV Infection,1.0,"""Sorry to say this drug Abacavir definitely had a very bad unpleasant stressful adverse effect felt in my heart it was beating harder and surroun..."
1,"September 1, 2019",Abacavir and lamivudine,HIV Infection,7.0,"""taking this combo for 2 months now together with rezolsta\n(darunavir/cobicistat)\nfeeling nausea dizzy have severe rash (not sure from meds tho)..."
2,"August 10, 2019",Abacavir and lamivudine,HIV Infection,10.0,"""My wife and I have B\been on Epzicom since 2000 along with viramune it works no side effects we take the generic. I don't understand why Drs..."
3,"August 16, 2013",Abacavir and lamivudine,HIV Infection,10.0,"""Epzicom Reyataz AND Norvir triple combination. It is expensive but ALL hiv medications are expensive and there are programs to help pay for the ..."
4,"March 16, 2013",Abacavir and lamivudine,HIV Infection,10.0,"""I've used this since first diagnosed and starting HAART in 2004 and it's worked extremely well: my numbers have never been below 1200 since 3 mo..."


In [8]:
print(df.date.min())
print(df.date.max())

April 1, 2008
September 9, 2019


In [24]:
df['comment'] = df.comment.str.replace('"', '')
df.head()

,date,drug_name,condition,rating,comment
0,"March 21, 2019",Abacavir,HIV Infection,1.0,Sorry to say this drug Abacavir definitely had...
1,"September 1, 2019",Abacavir and lamivudine,HIV Infection,7.0,taking this combo for 2 months now together wi...
2,"August 10, 2019",Abacavir and lamivudine,HIV Infection,10.0,My wife and I have B\been on Epzicom since 20...
3,"August 16, 2013",Abacavir and lamivudine,HIV Infection,10.0,Epzicom Reyataz AND Norvir triple combination....
4,"March 16, 2013",Abacavir and lamivudine,HIV Infection,10.0,I've used this since first diagnosed and start...


In [25]:
df = df[["rating", "comment"]]
df

,rating,comment
0,1.0,Sorry to say this drug Abacavir definitely had...
1,7.0,taking this combo for 2 months now together wi...
2,10.0,My wife and I have B\been on Epzicom since 20...
3,10.0,Epzicom Reyataz AND Norvir triple combination....
4,10.0,I've used this since first diagnosed and start...
...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...
175278,10.0,7 day treatment worked well on an infection th...
175279,6.0,Have had both the IV and oral versions of this...
175280,6.0,It didn't work as fast as I thought it would. ...


In [26]:
df["no_contract"] = df["comment"].apply(lambda x: [contractions.fix(word) for word in x.split()])
df

,rating,comment,no_contract
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit..."
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog..."
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ..."
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin..."
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ..."
...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ..."
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe..."
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions..."
175280,6.0,It didn't work as fast as I thought it would. ...,"[It, did not, work, as, fast, as, I, thought, ..."


In [27]:
df["comment_description_list"] = [' '.join(map(str, l)) for l in df["no_contract"]]
df

,rating,comment,no_contract,comment_description_list
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...",Sorry to say this drug Abacavir definitely had...
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...",taking this combo for 2 months now together wi...
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ...",My wife and I have B\been on Epzicom since 200...
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...",Epzicom Reyataz AND Norvir triple combination....
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ...",I have used this since first diagnosed and sta...
...,...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...",I had the MRSA virus i my thigh and it was the...
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...",7 day treatment worked well on an infection th...
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...",Have had both the IV and oral versions of this...
175280,6.0,It didn't work as fast as I thought it would. ...,"[It, did not, work, as, fast, as, I, thought, ...",It did not work as fast as I thought it would....


In [28]:
df["tokenized"] = df["comment_description_list"].apply(word_tokenize)
df

,rating,comment,no_contract,comment_description_list,tokenized
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...",Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit..."
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...",taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog..."
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ...",My wife and I have B\been on Epzicom since 200...,"[My, wife, and, I, have, B\been, on, Epzicom, ..."
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...",Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin..."
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ...",I have used this since first diagnosed and sta...,"[I, have, used, this, since, first, diagnosed,..."
...,...,...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...",I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ..."
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...",7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe..."
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...",Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions..."
175280,6.0,It didn't work as fast as I thought it would. ...,"[It, did not, work, as, fast, as, I, thought, ...",It did not work as fast as I thought it would....,"[It, did, not, work, as, fast, as, I, thought,..."


In [29]:
df["lower"] = df["tokenized"].apply(lambda x: [word.lower() for word in x])
df

,rating,comment,no_contract,comment_description_list,tokenized,lower
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...",Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit..."
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...",taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog..."
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ...",My wife and I have B\been on Epzicom since 200...,"[My, wife, and, I, have, B\been, on, Epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ..."
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...",Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin..."
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ...",I have used this since first diagnosed and sta...,"[I, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,..."
...,...,...,...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...",I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ..."
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...",7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe..."
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...",Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...","[have, had, both, the, iv, and, oral, versions..."
175280,6.0,It didn't work as fast as I thought it would. ...,"[It, did not, work, as, fast, as, I, thought, ...",It did not work as fast as I thought it would....,"[It, did, not, work, as, fast, as, I, thought,...","[it, did, not, work, as, fast, as, i, thought,..."


In [30]:
punc = string.punctuation
df["no_punc"] = df["lower"].apply(lambda x: [word for word in x if word not in punc])
df

,rating,comment,no_contract,comment_description_list,tokenized,lower,no_punc
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...",Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit..."
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...",taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog..."
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ...",My wife and I have B\been on Epzicom since 200...,"[My, wife, and, I, have, B\been, on, Epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ..."
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...",Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin..."
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ...",I have used this since first diagnosed and sta...,"[I, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,..."
...,...,...,...,...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...",I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ..."
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...",7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe..."
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...",Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...","[have, had, both, the, iv, and, oral, versions...","[have, had, both, the, iv, and, oral, versions..."
175280,6.0,It didn't work as fast as I thought it would. ...,"[It, did not, work, as, fast, as, I, thought, ...",It did not work as fast as I thought it would....,"[It, did, not, work, as, fast, as, I, thought,...","[it, did, not, work, as, fast, as, i, thought,...","[it, did, not, work, as, fast, as, i, thought,..."


In [31]:
stop_words = set(stopwords.words("english"))
df["stopwords_removed"] = df["no_punc"].apply(lambda x: [word for word in x if word not in stop_words])
df

,rating,comment,no_contract,comment_description_list,tokenized,lower,no_punc,stopwords_removed
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...",Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, say, drug, abacavir, definitely, bad, ..."
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...",taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, combo, 2, months, together, rezolsta,..."
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ...",My wife and I have B\been on Epzicom since 200...,"[My, wife, and, I, have, B\been, on, Epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[wife, b\been, epzicom, since, 2000, along, vi..."
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...",Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, norvir, triple, combination..."
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ...",I have used this since first diagnosed and sta...,"[I, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[used, since, first, diagnosed, starting, haar..."
...,...,...,...,...,...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...",I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[mrsa, virus, thigh, size, enlarged, grapefrui..."
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...",7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, infection, l..."
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...",Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...","[have, had, both, the, iv, and, oral, versions...","[have, had, both, the, iv, and, oral, versions...","[iv, oral, versions, antibiotic, hospital, two..."
175280,6.0,It didn't work as fast as I thought it would. ...,"[It, did not, work, as, fast, as, I, thought, ...",It did not work as fast as I thought it would....,"[It, did, not, work, as, fast, as, I, thought,...","[it, did, not, work, as, fast, as, i, thought,...","[it, did, not, work, as, fast, as, i, thought,...","[work, fast, thought, would, expensive, 3600, ..."


In [32]:
df["pos_tags"] = df["stopwords_removed"].apply(nltk.tag.pos_tag)
df

,rating,comment,no_contract,comment_description_list,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...",Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, say, drug, abacavir, definitely, bad, ...","[(sorry, JJ), (say, VBP), (drug, NN), (abacavi..."
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...",taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, combo, 2, months, together, rezolsta,...","[(taking, VBG), (combo, JJ), (2, CD), (months,..."
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ...",My wife and I have B\been on Epzicom since 200...,"[My, wife, and, I, have, B\been, on, Epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[wife, b\been, epzicom, since, 2000, along, vi...","[(wife, NN), (b\been, NN), (epzicom, NN), (sin..."
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...",Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, norvir, triple, combination...","[(epzicom, NN), (reyataz, NN), (norvir, JJ), (..."
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ...",I have used this since first diagnosed and sta...,"[I, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[used, since, first, diagnosed, starting, haar...","[(used, VBN), (since, IN), (first, JJ), (diagn..."
...,...,...,...,...,...,...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...",I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[mrsa, virus, thigh, size, enlarged, grapefrui...","[(mrsa, NN), (virus, NN), (thigh, JJ), (size, ..."
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...",7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, infection, l...","[(7, CD), (day, NN), (treatment, NN), (worked,..."
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...",Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...","[have, had, both, the, iv, and, oral, versions...","[have, had, both, the, iv, and, oral, versions...","[iv, oral, versions, antibiotic, hospital, two...","[(iv, JJ), (oral, JJ), (versions, NNS), (antib..."
175280,6.0,It didn't work as fast as I thought it would. ...,"[It, did not, work, as, fast, as, I, thought, ...",It did not work as fast as I thought it would....,"[It, did, not, work, as, fast, as, I, thought,...","[it, did, not, work, as, fast, as, i, thought,...","[it, did, not, work, as, fast, as, i, thought,...","[work, fast, thought, would, expensive, 3600, ...","[(work, NN), (fast, RB), (thought, VBD), (woul..."


In [33]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [34]:
df["wordnet_pos"] = df["pos_tags"].apply(lambda x: [(word, get_wordnet_pos(pos_tag))for (word, pos_tag) in x])
df

,rating,comment,no_contract,comment_description_list,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...",Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, say, drug, abacavir, definitely, bad, ...","[(sorry, JJ), (say, VBP), (drug, NN), (abacavi...","[(sorry, a), (say, v), (drug, n), (abacavir, n..."
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...",taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, combo, 2, months, together, rezolsta,...","[(taking, VBG), (combo, JJ), (2, CD), (months,...","[(taking, v), (combo, a), (2, n), (months, n),..."
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ...",My wife and I have B\been on Epzicom since 200...,"[My, wife, and, I, have, B\been, on, Epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[wife, b\been, epzicom, since, 2000, along, vi...","[(wife, NN), (b\been, NN), (epzicom, NN), (sin...","[(wife, n), (b\been, n), (epzicom, n), (since,..."
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...",Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, norvir, triple, combination...","[(epzicom, NN), (reyataz, NN), (norvir, JJ), (...","[(epzicom, n), (reyataz, n), (norvir, a), (tri..."
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ...",I have used this since first diagnosed and sta...,"[I, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[used, since, first, diagnosed, starting, haar...","[(used, VBN), (since, IN), (first, JJ), (diagn...","[(used, v), (since, n), (first, a), (diagnosed..."
...,...,...,...,...,...,...,...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...",I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[mrsa, virus, thigh, size, enlarged, grapefrui...","[(mrsa, NN), (virus, NN), (thigh, JJ), (size, ...","[(mrsa, n), (virus, n), (thigh, a), (size, n),..."
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...",7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, infection, l...","[(7, CD), (day, NN), (treatment, NN), (worked,...","[(7, n), (day, n), (treatment, n), (worked, v)..."
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...",Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...","[have, had, both, the, iv, and, oral, versions...","[have, had, both, the, iv, and, oral, versions...","[iv, oral, versions, antibiotic, hospital, two...","[(iv, JJ), (oral, JJ), (versions, NNS), (antib...","[(iv, a), (oral, a), (versions, n), (antibioti..."
175280,6.0,It didn't work as fast as I thought it would. ...,

In [35]:
wnl = WordNetLemmatizer()
df["lemmatized"] = df["wordnet_pos"].apply(lambda x: [wnl.lemmatize(word,tag) for word, tag in x])
df

,rating,comment,no_contract,comment_description_list,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,1.0,Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...",Sorry to say this drug Abacavir definitely had...,"[Sorry, to, say, this, drug, Abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, to, say, this, drug, abacavir, definit...","[sorry, say, drug, abacavir, definitely, bad, ...","[(sorry, JJ), (say, VBP), (drug, NN), (abacavi...","[(sorry, a), (say, v), (drug, n), (abacavir, n...","[sorry, say, drug, abacavir, definitely, bad, ..."
1,7.0,taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...",taking this combo for 2 months now together wi...,"[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, this, combo, for, 2, months, now, tog...","[taking, combo, 2, months, together, rezolsta,...","[(taking, VBG), (combo, JJ), (2, CD), (months,...","[(taking, v), (combo, a), (2, n), (months, n),...","[take, combo, 2, month, together, rezolsta, da..."
2,10.0,My wife and I have B\been on Epzicom since 20...,"[My, wife, and, I, have, B\been, on, Epzicom, ...",My wife and I have B\been on Epzicom since 200...,"[My, wife, and, I, have, B\been, on, Epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[my, wife, and, i, have, b\been, on, epzicom, ...","[wife, b\been, epzicom, since, 2000, along, vi...","[(wife, NN), (b\been, NN), (epzicom, NN), (sin...","[(wife, n), (b\been, n), (epzicom, n), (since,...","[wife, b\been, epzicom, since, 2000, along, vi..."
3,10.0,Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...",Epzicom Reyataz AND Norvir triple combination....,"[Epzicom, Reyataz, AND, Norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, and, norvir, triple, combin...","[epzicom, reyataz, norvir, triple, combination...","[(epzicom, NN), (reyataz, NN), (norvir, JJ), (...","[(epzicom, n), (reyataz, n), (norvir, a), (tri...","[epzicom, reyataz, norvir, triple, combination..."
4,10.0,I've used this since first diagnosed and start...,"[I have, used, this, since, first, diagnosed, ...",I have used this since first diagnosed and sta...,"[I, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[i, have, used, this, since, first, diagnosed,...","[used, since, first, diagnosed, starting, haar...","[(used, VBN), (since, IN), (first, JJ), (diagn...","[(used, v), (since, n), (first, a), (diagnosed...","[use, since, first, diagnose, start, haart, 20..."
...,...,...,...,...,...,...,...,...,...,...,...
175277,10.0,I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...",I had the MRSA virus i my thigh and it was the...,"[I, had, the, MRSA, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[i, had, the, mrsa, virus, i, my, thigh, and, ...","[mrsa, virus, thigh, size, enlarged, grapefrui...","[(mrsa, NN), (virus, NN), (thigh, JJ), (size, ...","[(mrsa, n), (virus, n), (thigh, a), (size, n),...","[mrsa, virus, thigh, size, enlarge, grapefruit..."
175278,10.0,7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...",7 day treatment worked well on an infection th...,"[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, on, an, infe...","[7, day, treatment, worked, well, infection, l...","[(7, CD), (day, NN), (treatment, NN), (worked,...","[(7, n), (day, n), (treatment, n), (worked, v)...","[7, day, treatment, work, well, infection, las..."
175279,6.0,Have had both the IV and oral versions of this...,"[Have, had, both, the, IV, and, oral, versions...",Have had both the IV and oral versions of th

In [36]:
df.to_pickle("df_clean.pkl")